# Crawl_AirPollutantData

In [1]:
import requests
import datetime
import threading
import time

|변수명|타입|변수설명|값설명|
|---|---|---|---|
|KEY|String(필수)|인증키|OpenAPI 에서 발급된 인증키|
|TYPE|String(필수)|요청파일타입|xml : xml, xml파일 : xmlf, 엑셀파일 : xls, json파일 : json|
|SERVICE|String(필수)|서비스명|TimeAverageAirQuality|
|START_INDEX|INTEGER(필수)|요청시작위치|정수 입력 (페이징 시작번호 입니다 : 데이터 행 시작번호)|
|END_INDEX|INTEGER(필수)|요청종료위치|정수 입력 (페이징 끝번호 입니다 : 데이터 행 끝번호)|
|MSRDT|STRING(필수)|측정일시|'YYYYMMDDHH24MI'|
|MSRSTE_NM|STRING(선택)|측정소명|문자열|

In [2]:
with open("./seoul_api_key.txt", "r") as f:
    api_key = f.read().replace("\n", "")

In [3]:
def get_json(url):
    response = requests.get(url)
    response_json = response.json()
    return pd.DataFrame(response_json["TimeAverageAirQuality"]["row"])
         
def get_pollution_data(start_date, end_date, api_key):
    dates = list(map(lambda x: x.replace("-", ""), list(pd.date_range(start_date, end_date).astype(str))))
    urls = ["http://openapi.seoul.go.kr:8088/{}/json/TimeAverageAirQuality/1/1000/{}".format(api_key, date) for date in dates]
    th_urls = [
        urls[:len(urls)//6],
        urls[len(urls)//6:2*(len(urls)//6)],
        urls[2*(len(urls)//6):3*(len(urls)//6)],
        urls[3*(len(urls)//6):4*(len(urls)//6)],
        urls[4*(len(urls)//6):5*(len(urls)//6)],
        urls[5*(len(urls)//6):],
    ]
    
    pollution_ls = []
    def crawling(urls):
        for url in urls:
            pollution_ls.append(get_json(url))

    
    for th_url in th_urls:
        th = threading.Thread(target=crawling, args = (th_url,))
        th.start()

    count = sec = 0
    while count < len(urls):
        time.sleep(5)
        sec += 5
        count = len(pollution_ls)
        print("{}초 경과. 수집된 데이터 수: {}".format(sec, count))      
    df = pd.concat(pollution_ls, sort = False)
    df["MSRDT"] = df["MSRDT"].apply(lambda date : datetime.datetime.strptime(str(date), "%Y%m%d%H%M"))
    df.sort_values(by = ["MSRDT"], inplace = True)
    return df[["MSRDT", "MSRSTE_NM", "NO2", "O3", "CO", "PM10", "PM25", "SO2"]].reset_index(drop=True)

In [4]:
start_date, end_date = "2015-09-19", "2019-05-31"
df_pollution = get_pollution_data(start_date, end_date, api_key)

5초 경과. 수집된 데이터 수: 254
10초 경과. 수집된 데이터 수: 475
15초 경과. 수집된 데이터 수: 698
20초 경과. 수집된 데이터 수: 946
25초 경과. 수집된 데이터 수: 1188
30초 경과. 수집된 데이터 수: 1334
35초 경과. 수집된 데이터 수: 1351


In [5]:
df_pollution.to_csv("../data/raw/dust_hour.csv", encoding = "euc-kr", index = False)

In [6]:
df_pollution.dtypes

MSRDT        datetime64[ns]
MSRSTE_NM            object
NO2                 float64
O3                  float64
CO                  float64
PM10                float64
PM25                float64
SO2                 float64
dtype: object